# Import the packages needed for the analysis below

In [2]:
# Import the packages
import pandas as pd
import numpy as np
from datetime import datetime
import missingno as mno #helps visualize Missing data
import os
from fuzzywuzzy import process
from datetime import datetime
import matplotlib.pyplot as plt  # plotting series
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA as arima
from statsmodels.tsa.arima_model import ARIMAResults as arima_results
from math import sqrt

Next read in the data from Zillow

In [3]:
#Import and inspect the data. Ask the professor if the link is working correctly
zillowdf = pd.read_csv("Zip_Zhvi_SingleFamilyResidence.csv")
#df = pd.read_csv('https://query.data.world/s/zmobklxwywreevgcfd5tjra6jp66wk')
zillowdf.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,1413747.0,1405862.0,1402547.0,1390420.0,1381621.0,1375725.0,1374714.0,1381453.0,1385737.0,1389268.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,364892.0,...,974693.0,975616.0,975734.0,975251.0,974238.0,973104.0,971908.0,972038.0,973671.0,975642.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,1528603.0,1514894.0,1502233.0,1492429.0,1486122.0,1480426.0,1476509.0,1478980.0,1479301.0,1474994.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,200475.0,...,335536.0,335878.0,335940.0,336092.0,336119.0,336083.0,336154.0,335860.0,336037.0,336483.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,546663.0,...,1207765.0,1208853.0,1208481.0,1206304.0,1204013.0,1201182.0,1198879.0,1198277.0,1199900.0,1200980.0


In [ ]:
# Check for missing values
mno.heatmap(zillowdf)

Error: Session cannot generate requests

In [10]:
#Make the dataset log
zillowdf = zillowdf.melt(id_vars=['RegionType','RegionID','RegionName','City','State','StateName','Metro','CountyName','SizeRank'],var_name="Period", value_name="MedianPrice")
zillowdf.head()

,RegionType,RegionID,RegionName,City,State,StateName,Metro,CountyName,SizeRank,Period,MedianPrice
0,Zip,61639,10025,New York,NY,NY,New York-Newark-Jersey City,New York County,0,1996-01-31,NaN
1,Zip,84654,60657,Chicago,IL,IL,Chicago-Naperville-Elgin,Cook County,1,1996-01-31,364892.0
2,Zip,61637,10023,New York,NY,NY,New York-Newark-Jersey City,New York County,2,1996-01-31,NaN
3,Zip,91982,77494,Katy,TX,TX,Houston-The Woodlands-Sugar Land,Harris County,3,1996-01-31,200475.0
4,Zip,84616,60614,Chicago,IL,IL,Chicago-Naperville-Elgin,Cook County,4,1996-01-31,546663.0


In [11]:
medianReview = zillowdf.pivot_table(index='Period', columns='RegionName', values='MedianPrice').reset_index()
medianReview.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Columns: 30465 entries, Period to 99929
dtypes: float64(30464), object(1)
memory usage: 67.6+ MB


In [12]:
medianReview['Period']=medianReview['Period'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
medianReview.set_index('Period', inplace=True)
medianReview.interpolate(method='time', limit_direction='backward',axis=0, inplace=True)
medianReview = medianReview.reset_index()
medianReview = medianReview.melt(id_vars='Period', var_name='ZipCode', value_name='MedianPrice')
medianReview.head()


,Period,ZipCode,MedianPrice
0,1996-01-31,501,68555.0
1,1996-02-29,501,68555.0
2,1996-03-31,501,68555.0
3,1996-04-30,501,68555.0
4,1996-05-31,501,68555.0


In [13]:
zillowdf=zillowdf[['RegionName','Metro', 'State']]
zillowdf=zillowdf.drop_duplicates()
zillowdf = zillowdf.merge(medianReview, how='left', left_on='RegionName', right_on='ZipCode')
zillowdf['Metro'] = zillowdf['Metro']+", "+zillowdf['State']
del medianReview

In [14]:
zillowdf['Year'] = zillowdf.Period.dt.year
zillowdf=zillowdf.groupby(['Year','Metro']).agg('mean').reset_index()
zillowdf.dropna(inplace=True)

In [15]:
zillowdf.head()

,Year,Metro,MedianPrice
0,1996,"Aberdeen, SD",111537.133333
1,1996,"Aberdeen, WA",123781.368421
2,1996,"Abilene, TX",85570.434783
3,1996,"Ada, OK",43909.500000
4,1996,"Adrian, MI",95243.259804


# Bureau of Labor Statistics

In [16]:
#This block calls in the burea of labor statistics data
# Removes extra spaces from the field name for series id and keep only the cloumns needed for the analysis
# Removes extra spaces from the series ids themselves and lastly inspects the data
blsSeries = pd.read_csv('la.series', sep="\t")
blsSeries.columns
blsSeries.rename(columns={'series_id                     ':'series_id'}, inplace=True)
blsSeries = blsSeries[['series_id','series_title']]
blsSeries['series_id']=blsSeries['series_id'].apply(lambda x: x.strip()) #cleans up extra spaces from series ID
blsSeries.head()

,series_id,series_title
0,LASBS060000000000003,"Unemployment Rate: Balance of California, stat..."
1,LASBS060000000000004,"Unemployment: Balance of California, state les..."
2,LASBS060000000000005,"Employment: Balance of California, state less ..."
3,LASBS060000000000006,"Labor Force: Balance of California, state less..."
4,LASBS120000000000003,"Unemployment Rate: Balance of Florida, state l..."


In [17]:
#this calls in the data from each series and merges it with the series in formation so the information is readable
blsData = pd.read_csv('laborstats.txt', sep='\t')
blsData = blsSeries.merge(blsData, how='right', left_on='series_id', right_on='Series ID')
del blsSeries #This file is no longer needed

C:\Users\kyles\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
blsData.head()

,series_id,series_title,Series ID,Jan 1996,Feb 1996,Mar 1996,Apr 1996,May 1996,Jun 1996,Jul 1996,...,Mar 2017,Apr 2017,May 2017,Jun 2017,Jul 2017,Aug 2017,Sep 2017,Oct 2017,Nov 2017,Dec 2017
0,LAUMC014282000000003,"Unemployment Rate: Selma, AL Micropolitan Stat...",LAUMC014282000000003,11.7,12.4,11.4,10.4,11.5,14.1,13.6,...,8.1,6.8,6.7,9.0,9.7,8.2,7.3,6.3,6.1,6.1
1,LAUMC014282000000004,"Unemployment: Selma, AL Micropolitan Statistic...",LAUMC014282000000004,2360.0,2511.0,2291.0,2054.0,2292.0,2930.0,2818.0,...,1226.0,1030.0,1000.0,1362.0,1469.0,1220.0,1103.0,944.0,895.0,901.0
2,LAUMC014282000000005,"Employment: Selma, AL Micropolitan Statistical...",LAUMC014282000000005,17808.0,17755.0,17800.0,17777.0,17697.0,17820.0,17850.0,...,13827.0,14035.0,13944.0,13712.0,13720.0,13721.0,13935.0,14060.0,13889.0,13751.0
3,LAUMC014282000000006,"Labor Force: Selma, AL Micropolitan Statistica...",LAUMC014282000000006,20168.0,20266.0,20091.0,19831.0,19989.0,20750.0,20668.0,...,15053.0,15065.0,14944.0,15074.0,15189.0,14941.0,15038.0,15004.0,14784.0,14652.0
4,LAUMC014518000000003,"Unemployment Rate: Talladega-Sylacauga, AL Mic...",LAUMC014518000000003,8.3,8.2,7.1,6.8,7.5,7.8,7.5,...,5.7,4.8,4.7,5.4,5.3,4.8,4.3,4.4,4.2,4.1


In [19]:
# Split the series title into measure and label
split = blsData['series_title'].str.split(": ", n=1, expand=True)
split

,0,1
0,Unemployment Rate,"Selma, AL Micropolitan Statistical Area (U)"
1,Unemployment,"Selma, AL Micropolitan Statistical Area (U)"
2,Employment,"Selma, AL Micropolitan Statistical Area (U)"
3,Labor Force,"Selma, AL Micropolitan Statistical Area (U)"
4,Unemployment Rate,"Talladega-Sylacauga, AL Micropolitan Statistic..."
...,...,...
3743,Labor Force,"Casper, WY Metropolitan Statistical Area (U)"
3744,Unemployment Rate,"Cheyenne, WY Metropolitan Statistical Area (U)"
3745,Unemployment,"Cheyenne, WY Metropolitan Statistical Area (U)"
3746,Employment,"Cheyenne, WY Metropolitan Statistical Area (U)"


In [20]:
#Add the new columns to the data frame
blsData['Measure']=split[0]
blsData['Metro'] = split[1]
blsData

,series_id,series_title,Series ID,Jan 1996,Feb 1996,Mar 1996,Apr 1996,May 1996,Jun 1996,Jul 1996,...,May 2017,Jun 2017,Jul 2017,Aug 2017,Sep 2017,Oct 2017,Nov 2017,Dec 2017,Measure,Metro
0,LAUMC014282000000003,"Unemployment Rate: Selma, AL Micropolitan Stat...",LAUMC014282000000003,11.7,12.4,11.4,10.4,11.5,14.1,13.6,...,6.7,9.0,9.7,8.2,7.3,6.3,6.1,6.1,Unemployment Rate,"Selma, AL Micropolitan Statistical Area (U)"
1,LAUMC014282000000004,"Unemployment: Selma, AL Micropolitan Statistic...",LAUMC014282000000004,2360.0,2511.0,2291.0,2054.0,2292.0,2930.0,2818.0,...,1000.0,1362.0,1469.0,1220.0,1103.0,944.0,895.0,901.0,Unemployment,"Selma, AL Micropolitan Statistical Area (U)"
2,LAUMC014282000000005,"Employment: Selma, AL Micropolitan Statistical...",LAUMC014282000000005,17808.0,17755.0,17800.0,17777.0,17697.0,17820.0,17850.0,...,13944.0,13712.0,13720.0,13721.0,13935.0,14060.0,13889.0,13751.0,Employment,"Selma, AL Micropolitan Statistical Area (U)"
3,LAUMC014282000000006,"Labor Force: Selma, AL Micropolitan Statistica...",LAUMC014282000000006,20168.0,20266.0,20091.0,19831.0,19989.0,20750.0,20668.0,...,14944.0,15074.0,15189.0,14941.0,15038.0,15004.0,14784.0,14652.0,Labor Force,"Selma, AL Micropolitan Statistical Area (U)"
4,LAUMC014518000000003,"Unemployment Rate: Talladega-Sylacauga, AL Mic...",LAUMC014518000000003,8.3,8.2,7.1,6.8,7.5,7.8,7.5,...,4.7,5.4,5.3,4.8,4.3,4.4,4.2,4.1,Unemployment Rate,"Talladega-Sylacauga, AL Micropolitan Statistic..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3743,LAUMT561622000000006,"Labor Force: Casper, WY Metropolitan Statistic...",LAUMT561622000000006,32743.0,32702.0,32948.0,32431.0,32142.0,32218.0,32298.0,...,39510.0,39600.0,39638.0,39542.0,39589.0,39732.0,40309.0,40009.0,Labor Force,"Casper, WY Metropolitan Statistical Area (U)"
3744,LAUMT561694000000003,"Unemployment Rate: Cheyenne, WY Metropolitan S...",LAUMT561694000000003,4.8,4.0,4.1,3.7,3.2,3.3,3.3,...,3.5,3.9,3.6,3.6,3.6,3.6,3.7,3.7,Unemployment Rate,"Cheyenne, WY Metropolitan Statistical Area (U)"
3745,LAUMT561694000000004,"Unemployment: Cheyenne, WY Metropolitan Statis...",LAUMT561694000000004,1883.0,1542.0,1617.0,1454.0,1225.0,1272.0,1293.0,...,1689.0,1893.0,1784.0,1737.0,1714.0,1736.0,1825.0,1768.0,Unemployment,"Cheyenne, WY Metropolitan Statistical Area (U)"
3746,LAUMT561694000000005,"Employment: Cheyenne, WY Metropolitan Statisti...",LAUMT561694000000005,37141.0,37357.0,37464.0,37352.0,37233.0,37370.0,38106.0,...,46649.0,46272.0,47192.0,46410.0,46414.0,46345.0,46887.0,46499.0,Employment,"Cheyenne, WY Metropolitan Statistical Area (U)"


# FBI Crime and Population Data

Importing the data about Crime in the specific metropolitan regions from the FBI. The data is only available for about a decade and by year so will need to think about how to adjust the model

In [21]:
#Start by changing the directory to where the files are saved and capturing consitent column names which always come in this patter, but very slightly
# in their text expression
os.chdir('crime')
files = os.listdir()
colNames = ['Metropolitan Statistical Area', 'Counties/principal cities',
       'Population', 'Violent crime', 'Murder and nonnegligent manslaughter',
       'Forcible rape', 'Robbery', 'Aggravated assault', 'Property crime',
       'Burglary', 'Larceny-theft', 'Motor vehicle theft']

In [22]:
# Next I'll read in the data and combine into a single dataframe
# this hould spit out a review of the dataframe
for file in files:
    df = pd.read_excel(file, skiprows=3)
    df = df.drop(df.iloc[:,12:],axis=1)
    cols = dict(zip(df.columns, colNames))
    df.rename(columns=cols, inplace=True)
    df['Metropolitan Statistical Area'].ffill(axis=0, inplace=True)
    df['Counties/principal cities'] = df['Counties/principal cities'].fillna('Population')
    # Filter the rows for population and crime per 100,000
    df=df[(df['Counties/principal cities']=='Population')|(df['Counties/principal cities']=='Rate per 100,000 inhabitants')]
    #Fill the population data into the rate per 100,000
    df['Population'].ffill(axis=0,inplace=True)
    df=df.dropna(axis = 0, subset=['Violent crime','Property crime'])
    df = df.drop(df.iloc[:,12:],axis=1)
    df['year']=int(file[:4])
    df=df[df['Counties/principal cities']=='Rate per 100,000 inhabitants']
    try:
        crimeData = crimeData.append(df, ignore_index=True)
    except:
        crimeData = df

crimeData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4266 entries, 0 to 4265
Data columns (total 13 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Metropolitan Statistical Area         4266 non-null   object
 1   Counties/principal cities             4266 non-null   object
 2   Population                            4266 non-null   object
 3   Violent crime                         4266 non-null   object
 4   Murder and nonnegligent manslaughter  4266 non-null   object
 5   Forcible rape                         4266 non-null   object
 6   Robbery                               4266 non-null   object
 7   Aggravated assault                    4266 non-null   object
 8   Property crime                        4266 non-null   object
 9   Burglary                              4266 non-null   object
 10  Larceny-theft                         4266 non-null   object
 11  Motor vehicle theft           

In [23]:
crimeData.head()

,Metropolitan Statistical Area,Counties/principal cities,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,year
0,"Abilene, TX M.S.A.1","Rate per 100,000 inhabitants",162776,391.9,3.7,46.1,67,275.2,3526.9,940.6,2366.4,219.9,2006
1,"Albany, GA M.S.A.","Rate per 100,000 inhabitants",168071,402.2,6.5,22,157.7,216,4266.6,1286.4,2673.9,306.4,2006
2,"Albany-Schenectady-Troy, NY M.S.A.","Rate per 100,000 inhabitants",851151,371.8,2.1,24.3,113.3,232.2,2743.2,569.7,2034.2,139.3,2006
3,"Albuquerque, NM M.S.A.","Rate per 100,000 inhabitants",808790,777.8,8.9,51.9,174.3,542.7,4824.9,1109.3,2885.3,830.3,2006
4,"Alexandria, LA M.S.A.","Rate per 100,000 inhabitants",140250,1067.4,7.1,52,154,854.2,5246.3,1533.7,3329.1,383.6,2006


# Fuzzy Text Match
Next we'll align the metro groups using Fuzzywuzzy package. Since the metro names vary slightly in each group.

In [24]:
blsmetro=blsData['Metro'].unique()
zillowMetro=zillowdf['Metro'].unique()
crimeMetro = crimeData['Metropolitan Statistical Area'].unique()
print(len(blsmetro))
print(len(zillowMetro))
print(len(crimeMetro))

937
1006
790


In [25]:
blsData['FuzzyMetro'] = blsData['Metro'].apply(lambda x: process.extractOne(x, zillowMetro))
blsData.head()

,series_id,series_title,Series ID,Jan 1996,Feb 1996,Mar 1996,Apr 1996,May 1996,Jun 1996,Jul 1996,...,Jun 2017,Jul 2017,Aug 2017,Sep 2017,Oct 2017,Nov 2017,Dec 2017,Measure,Metro,FuzzyMetro
0,LAUMC014282000000003,"Unemployment Rate: Selma, AL Micropolitan Stat...",LAUMC014282000000003,11.7,12.4,11.4,10.4,11.5,14.1,13.6,...,9.0,9.7,8.2,7.3,6.3,6.1,6.1,Unemployment Rate,"Selma, AL Micropolitan Statistical Area (U)","(Selma, AL, 90)"
1,LAUMC014282000000004,"Unemployment: Selma, AL Micropolitan Statistic...",LAUMC014282000000004,2360.0,2511.0,2291.0,2054.0,2292.0,2930.0,2818.0,...,1362.0,1469.0,1220.0,1103.0,944.0,895.0,901.0,Unemployment,"Selma, AL Micropolitan Statistical Area (U)","(Selma, AL, 90)"
2,LAUMC014282000000005,"Employment: Selma, AL Micropolitan Statistical...",LAUMC014282000000005,17808.0,17755.0,17800.0,17777.0,17697.0,17820.0,17850.0,...,13712.0,13720.0,13721.0,13935.0,14060.0,13889.0,13751.0,Employment,"Selma, AL Micropolitan Statistical Area (U)","(Selma, AL, 90)"
3,LAUMC014282000000006,"Labor Force: Selma, AL Micropolitan Statistica...",LAUMC014282000000006,20168.0,20266.0,20091.0,19831.0,19989.0,20750.0,20668.0,...,15074.0,15189.0,14941.0,15038.0,15004.0,14784.0,14652.0,Labor Force,"Selma, AL Micropolitan Statistical Area (U)","(Selma, AL, 90)"
4,LAUMC014518000000003,"Unemployment Rate: Talladega-Sylacauga, AL Mic...",LAUMC014518000000003,8.3,8.2,7.1,6.8,7.5,7.8,7.5,...,5.4,5.3,4.8,4.3,4.4,4.2,4.1,Unemployment Rate,"Talladega-Sylacauga, AL Micropolitan Statistic...","(Talladega-Sylacauga, AL, 90)"


In [26]:
crimeData['FuzzyMetro'] = crimeData['Metropolitan Statistical Area'].apply(lambda x: process.extractOne(x, zillowMetro))
crimeData['FuzzyMetro'].value_counts()

(Bakersfield, CA, 86)                 71
(Arcadia, FL, 86)                     61
(Adrian, MI, 86)                      41
(Chico, CA, 60)                       38
(Abilene, TX, 86)                     30
                                      ..
(Montgomery, AL, 90)                   1
(Tuscaloosa, AL, 90)                   1
(Chicago-Naperville-Elgin, IL, 95)     1
(Anchorage, AK, 90)                    1
(Las Cruces, NM, 90)                   1
Name: FuzzyMetro, Length: 478, dtype: int64